In [1]:
import pickle
import numpy as np 
import pandas as pd 
from sklearn import cross_validation
from sklearn.metrics import accuracy_score

In [3]:
%time data = pickle.load(open('training_set_3', 'r+'))

CPU times: user 2.13 s, sys: 758 ms, total: 2.88 s
Wall time: 2.92 s


In [ ]:
page_number, freq = 0, 200

In [4]:
from xgboost import XGBRegressor
from sklearn.linear_model import Lasso, BayesianRidge
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import GradientBoostingRegressor

xgboost = XGBRegressor(max_depth=10, learning_rate=0.1, n_estimators=30, nthread=-1, gamma=0, min_child_weight=1)

In [ ]:
# ****************************************************************************#
#    Exhaustive grid search
#    10-fold cross validation, xgboost, training seperately
# ****************************************************************************#
from sklearn import svm
from sklearn import grid_search
from sklearn import grid_search as gs
from sklearn import cross_validation 
from sklearn.metrics import make_scorer
import datetime as dt 
@make_scorer
def scorer(y, y_pred, **kwargs):
    return sum(abs((y_pred - y)/(y_pred + y)))/len(y_pred)

model = XGBRegressor()
param_grid = [
    {'max_depth': range(6,20,3), 'n_estimators': range(10,100,10)},
]
xgb_model = gs.GridSearchCV(model, param_grid, cv = 10, scoring=scorer)

start_time = dt.datetime.now()
best_models = dict()
for shop_id in range(1, 2001):
    dataset = data[data.shop_id==shop_id].loc[:,map(lambda x:True if x != 'date' else False, data.columns.tolist())]
    xgb_model.fit(dataset.iloc[:,:-1],dataset.iloc[:,-1])
    scores = xgb_model.grid_scores_
    sorted_index = sorted(np.arange(0, len(scores)), key = lambda x:scores[x].mean_validation_score, reverse = False)
    best_model_index = sorted_index[0]
    best_model = {
            'model': scores[best_model_index].parameters,
            'mean_loss': scores[best_model_index].mean_validation_score,
             'cvScores': [score for score in scores[best_model_index].cv_validation_scores]
    }
    best_models[shop_id] = best_model
end_time = dt.datetime.now()

print '耗时：{}'.format(end_time - start_time)

In [63]:
#****************************************************************************#
#    calculate the best model parameters
#****************************************************************************#

{'model': {'n_estimators': 30, 'max_depth': 6}, 'cvScores': [0.13738090030389954, 0.12994805838572657, 0.073383144869004407, 0.071120030505917795, 0.071134932598169359, 0.054616553033857929, 0.067954375742384429, 0.059462328098182665, 0.057027285735110025, 0.055669496056171407], 'mean_loss': 0.077927720924855504}


In [54]:
scores[0].mean_validation_score

0.19882586378037012

In [44]:
dir(scores[0])

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getnewargs__',
 '__getslice__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '_asdict',
 '_fields',
 '_make',
 '_replace',
 'count',
 'cv_validation_scores',
 'index',
 'mean_validation_score',
 'parameters']